In [1]:
function add_kbn(s, c, a)
    t = s + a
    d = c + ifelse(abs(s) ≥ abs(a), (s-t) + a, (a-t) + s)
    t, d
end

function sum_kbn(f, iter; T = Float64, s = zero(T), c = zero(T))
    for n in iter
        s, c = add_kbn(s, c, f(T(n)))
    end
    s + c
end

sum_kbn(iter; T = Float64, s = zero(T), c = zero(T)) =
    sum_kbn(identity, iter; T, s, c)

function sumupto_kbn(f, iter, x; T = Float64, s = zero(T), c = zero(T))
    for n in iter
        s, c = add_kbn(s, c, f(T(n)))
        s + c ≥ x && return s + c, n
    end
    s + c, typemax(eltype(iter))
end

@time sum_kbn(inv, 1:10^9)

  1.627475 seconds


21.300481502347942

In [2]:
using SpecialFunctions, Printf
H(n) = digamma(n+1) + MathConstants.γ
@printf "%.15f" H(big(10^9))

21.300481502347944

In [3]:
@time s = sum_kbn(inv, 1:7*10^8)
@time h, n = sumupto_kbn(inv, 7*10^8+1:10^9, 21.0; s)

  1.144573 seconds (1 allocation: 16 bytes)
  0.100277 seconds (45.48 k allocations: 2.722 MiB)


(21.000000000401332, 740461601)

In [4]:
@printf "%.15f" H(big(n))

21.000000000401333

In [5]:
@show Threads.nthreads()
using Distributed

function sum_kbn_threads(f, ran; T = Float64, nth = Threads.nthreads())
    splitran = Distributed.splitrange(ran[begin], ran[end], nth)
    S, C = zeros(T, nth), zeros(T, nth)
    Threads.@threads for i in 1:nth
        S[i] = sum_kbn(f, splitran[i]; T)
    end
    S .+= C
    S
end

Threads.nthreads() = 12


sum_kbn_threads (generic function with 1 method)

In [6]:
@time S = sum_kbn_threads(inv, 1:10^9)
sum_kbn(S)

  0.406791 seconds (323.65 k allocations: 19.816 MiB, 15.79% compilation time)


21.300481502347942

In [7]:
m, x = 7*10^8, 21.0
@time S = sum_kbn_threads(inv, 1:m)
@time s = sum_kbn(S)
@time h, n = sumupto_kbn(inv, m+1:round(Int, 1.1*m), x; s)

  0.212324 seconds (65.85 k allocations: 3.753 MiB, 4.37% compilation time)
  0.000005 seconds (3 allocations: 48 bytes)
  0.081589 seconds (64 allocations: 3.250 KiB, 3.56% compilation time)


(21.000000000401336, 740461601)

In [8]:
@printf "%.15f" H(big(n))

21.000000000401333

In [9]:
@time S = sum_kbn_threads(inv, 1:6*10^9)
sum_kbn(S)

  1.706449 seconds (115 allocations: 8.016 KiB)


23.09224097115933

In [10]:
m, x = 6*10^12, 30.0
@time S = sum_kbn_threads(inv, 1:m)
@time s = sum_kbn(S)
@time h, n = sumupto_kbn(inv, m+1:round(Int, 1.1*m), x; s)

1688.581247 seconds (118 allocations: 8.109 KiB)
  0.000012 seconds (3 allocations: 48 bytes)
  0.044741 seconds (18 allocations: 400 bytes)


(30.00000000000002, 6000022499693)

In [11]:
@printf "%.15f" H(big(n))

30.000000000000022